# 关于 Token


## 关于 BEP

BPE 每一步都将最常见的一对相邻数据单位替换为该数据中没有出现过的一个新单位，反复迭代直到满足停止条件。
假设我们有需要编码（压缩）的数据 aaabdaaabac。相邻字节对（相邻数据单位在 BPE 中看作相邻字节对） aa 最常出现，因此我们将用一个新字节 Z 替换它。我们现在有了 ZabdZabac，其中 Z = aa。下一个常见的字节对是 ab，让我们用 Y 替换它。我们现在有 ZYdZYac，其中 Z = aa ，Y = ab。剩下的唯一字节对是 ac，它只有一个，所以我们不对它进行编码。我们可以递归地使用字节对编码将 ZY 编码为 X。我们的数据现在已转换为 XdXac，其中 X = ZY，Y = ab，Z = aa。它不能被进一步压缩，因为没有出现多次的字节对。那如何把压缩的编码复原呢？反向执行以上过程就行了。

NLP 实例

NLP 中使用了上述算法的一个变体。光听不练你永远无法掌握精髓，让我们通过一个实际的例子来了解一下它的 NLP 版本。

首先来明确一下基础概念：token 可以理解为一个符号，就代表一个语言单位（就像单词，字符等）；tokenize 的意思是把一个句子或长语料分成 token。

BPE 确保最常见的词在 token 列表中表示为单个 token，而罕见的词被分解为两个或多个 subword tokens（不知道中文怎么翻译这个词 :-c，子词单位？emm），因此 BPE 也是典型的基于 subword 的 tokenization 算法。

假设我们有一个语料库，其中包含单词（pre-tokenization 之后）—— old, older, highest, 和 lowest，我们计算这些词在语料库中的出现频率。假设这些词出现的频率如下：

{“old”: 7, “older”: 3, “finest”: 9, “lowest”: 4}

让我们在每个单词的末尾添加一个特殊的结束标记“</w>”。

{“old</w>”: 7, “older</w>”: 3, “finest</w>”: 9, “lowest</w>”: 4}

在每个单词的末尾添加“</w>”标记以标识单词边界能够让算法知道每个单词的结束位置（因为我们统计相邻字符对时不能把分别位于两个单词中的字符对算进去），这有助于算法查看每个字符并找到频率最高的字符配对。稍后我们将看到“</w>”也能被算作字符对的一部分。

BPE 算法的下一步是寻找最频繁的字符对，合并它们，并一次又一次地执行相同的迭代，直到达到我们预先设置的 token 数限制或迭代限制。

合并字符可以让你用最少的 token 来表示语料库，这也是 BPE 算法的主要目标，即数据的压缩。为了合并，BPE 寻找最常出现的字节对。在这里，我们将字符视为与字节等价。当然，这只是英语的用法，其他语言可能有所不同。现在我们将最常见的字节对合并成一个 token，并将它们添加到 token 列表中，并重新计算每个 token 出现的频率。这意味着我们的频率计数将在每个合并步骤后发生变化。我们将继续执行此合并步骤，直到达到我们预先设置的 token 数限制或迭代限制。

## TikToken

TikToken 是 OpenAI 开发的一个工具，用于从文本中计算 token 的数量，而不实际执行任何编码。这对于估计使用 GPT-3 或其他 OpenAI API 的成本非常有用，因为这些 API 的计费通常是基于 token 数量的。

TikToken 的算法基于 Byte-Pair Encoding (BPE)。BPE 是一种常见的 NLP 任务中使用的文本编码方法。TikToken 使用 BPE 来估计文本中的 token 数量。

简单来说，TikToken 的工作原理如下：

- 预处理：文本首先被分割成词汇或子词。
- 应用 BPE 规则：使用预先定义的 BPE 规则将文本分割成 tokens。
- 计数 tokens：计算得到的 tokens 的数量。

这种方法的优点是，它可以快速估计 token 的数量，而不需要实际调用 API 或执行完整的编码过程。


In [1]:
%pip install tiktoken

  Using cached tiktoken-0.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 9.2 kB/s eta 0:00:0031m153.6 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.4/924.4 kB 10.7 kB/s eta 0:00:00m eta 0:00:010:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 18.2 kB/s eta 0:00:0031m18.0 kB/s eta 0:00:02
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 使用方法

In [2]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"
enc = tiktoken.encoding_for_model("gpt-4")